In [ ]:
!pip install datasets youtokentome unidecode --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import youtokentome as yttm
from tqdm.autonotebook import tqdm
from unidecode import unidecode
import re

In [ ]:
wmt16 = load_dataset("wmt16", "de-en")

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

In [ ]:
wmt16['train'] = wmt16['train'].select(range(1_150_000))

In [ ]:
wmt_train = wmt16['train']['translation']
wmt_val = wmt16['validation']['translation']
wmt_test = wmt16['test']['translation']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data_en_path = '/content/drive/MyDrive/pet_project/truncated_en_sentences.txt'
train_data_de_path = '/content/drive/MyDrive/pet_project/truncated_de_sentences.txt'

tokenizer_en_path = '/content/drive/MyDrive/pet_project/truncated_en_tokenizer.tok'
tokenizer_de_path = '/content/drive/MyDrive/pet_project/truncated_de_tokenizer.tok'

In [ ]:
# удалить все знаки препинания из предлжения, т.к. bpe будет распознавать в т.ч. вместе в точками и не только (но попробовать можно пока и так)

with open(train_data_en_path, 'w+') as en:
    with open(train_data_de_path, 'w+') as fr:
        for name in [wmt_train, wmt_val, wmt_test]:
            for el in tqdm(name):
                en.write(re.sub(r'[^\w\s]','',el['en'].lower()) + '\n')
                fr.write(re.sub(r'[^\w\s]','',unidecode(el['de'].lower())) + '\n') # normalize('NFKD', word)

  0%|          | 0/1150000 [00:00<?, ?it/s]

  0%|          | 0/2169 [00:00<?, ?it/s]

  0%|          | 0/2999 [00:00<?, ?it/s]

In [ ]:
PAD_token = 0
BOS_token = 1
EOS_token = 2
UNK_token = 3

In [ ]:
yttm.BPE.train(
    data=train_data_de_path,
    vocab_size=10_000,
    model=tokenizer_de_path,
    pad_id=PAD_token,
    unk_id=UNK_token,
    bos_id=BOS_token,
    eos_id=EOS_token
)

In [ ]:
yttm.BPE.train(
    data=train_data_en_path,
    vocab_size=10_000,
    model=tokenizer_en_path,
    pad_id=PAD_token,
    unk_id=UNK_token,
    bos_id=BOS_token,
    eos_id=EOS_token
)

In [ ]:
tokenizer_de = yttm.BPE(model=tokenizer_de_path)

In [ ]:
def clear_text(sent):
    wo_prep = re.sub(r'[^\w\s]','', sent)
    return wo_prep.lower()

In [ ]:
idx_ = 5

In [ ]:
wmt_train[idx_]['de']

'Ich bitte Sie, sich zu einer Schweigeminute zu erheben.'

In [ ]:
de_sent = wmt_train[idx_]['de']

In [ ]:
ans = tokenizer_de.encode(
        sentences=[unidecode(clear_text(de_sent))],
        output_type=yttm.OutputType.ID,
        bos=True, eos=True
    )[0]

In [ ]:
sent = tokenizer_de.decode(ans)

In [ ]:
sent

['<BOS> ich bitte sie sich zu einer schweigeminute zu erheben<EOS>']